In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.7)

In [4]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Give a user profile with name, age, and city.")
chain = prompt | llm
response = chain.invoke({})

In [5]:
print(response.content)

Name: Sarah Johnson
Age: 28
City: New York City


### Str output parser

In [6]:
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [8]:
llm = ChatOpenAI(temperature=0.7)

C:\Users\HP\AppData\Local\Temp\ipykernel_18700\1060472456.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7)


In [9]:
prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")

In [10]:
chain = prompt | llm | StrOutputParser()

In [11]:
print(chain.invoke({"country": "India"}))

The capital of India is New Delhi.


In [12]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}.")
chain = prompt | llm | StrOutputParser()
print(chain.invoke({"topic": "cats"}))

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


### Json output parser

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [17]:
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()

In [22]:
prompt_template = ChatPromptTemplate.from_template(
    """Extract the country and capital from the following text about a country.
    Text: {country_info}
    {format_instructions}"""
)

In [23]:
prompt = prompt_template.partial(format_instructions=format_instructions)
chain = prompt | model | parser
country_info = "India is a country in south Asia. Its capital is New Delhi."
response = chain.invoke({"country_info": country_info})

In [24]:
print(response)

{'country': 'India', 'capital': 'New Delhi'}


In [25]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

parser = JsonOutputParser()
prompt = ChatPromptTemplate.from_template(
    "Give a JSON with name and age.\n{format_instructions}",
).partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI()
chain = prompt | llm | parser

print(chain.invoke({}))

{'name': 'Alice', 'age': 30}


### Structured Output Parser

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [29]:
model = ChatOpenAI(temperature=0.7)

In [31]:
response_schemas = [  
    ResponseSchema(name="name", description="The name of the person"),
    ResponseSchema(name="age", description="The age of the person"),
    ResponseSchema(name="city", description="The city of the person"),
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = parser.get_format_instructions()


prompt_template = ChatPromptTemplate.from_template(
    """Extract information about the person described below.
    Description: {person_description}
    {format_instructions}"""
)


prompt = prompt_template.partial(format_instructions=format_instructions)

In [32]:
chain = prompt | model | parser 
person_descripton = "Muskan is a 25- 30 year olde data scientist from india"
response = chain.invoke({"person_description": person_descripton})
print(response)

{'name': 'Muskan', 'age': '25-30', 'city': 'India'}


## Pydantic Output Parser

In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [34]:
from pydantic import BaseModel, Field
from typing import List, Optional

In [41]:
class Product(BaseModel):
    name : str = Field(description="The name of the product")
    price : float = Field(description="The price of the product")

parser = PydanticOutputParser(pydantic_object=Product, format_instructions="Return a JSON with name and price of the product")

In [42]:
prompt = ChatPromptTemplate.from_template(
    "Provide product details in JSON.\n{format_instructions}"
).partial(format_instructions=parser.get_format_instructions())

In [43]:
llm =ChatOpenAI()
chain = prompt | llm | parser

In [44]:
print(chain.invoke({}))

name='Smartphone' price=799.99


In [45]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel

class WeatherInfo(BaseModel):
    city: str
    temperature: float
    condition: str 


parser = PydanticOutputParser(pydantic_object=WeatherInfo)




prompt = PromptTemplate(
    template="Provide current weather info.\n{format_instructions}\nCity: {city}",
    input_variables=["city"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


llm = ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo")


chain = LLMChain(
    llm=llm,
    prompt=prompt,
 
)


In [46]:
city_input = "Bangalore"

raw_input= chain.run({"city": city_input})
print(raw_input)
print(parser.parse(raw_input))

C:\Users\HP\AppData\Local\Temp\ipykernel_18700\839694582.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_input= chain.run({"city": city_input})


{
  "city": "Bangalore",
  "temperature": 28,
  "condition": "Partly Cloudy"
}
city='Bangalore' temperature=28.0 condition='Partly Cloudy'


## Enum

In [47]:
from langchain.output_parsers import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from enum import Enum

In [48]:
class Color(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"



parser = EnumOutputParser(enum=Color)

prompt = PromptTemplate(
    template="Choose a color from red, green, or blue. Respond in lowercase only.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(temperature=0)

In [49]:
output = model.invoke(prompt.format())

parsed = parser.parse(output.content.lower()) 


print(f"Raw Output: {output.content}")
print(f"Parsed Output: {parsed}")

Raw Output: green
Parsed Output: Color.GREEN


In [50]:
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser()
prompt = PromptTemplate(
    template="What is the current date and time?\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(temperature=0)
output = model.invoke(prompt.format())
parsed = parser.parse(output.content)
print(parsed)

2022-01-06 15:45:30


In [51]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
    template="List three fruits.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
model = ChatOpenAI(temperature=0)
output = model.invoke(prompt.format())

parsed = parser.parse(output.content)
print(parsed)

['apple', 'banana', 'orange']


In [52]:
print("The End")

The End
